In [1]:
import imageio
import time
import math
import cv2
import numpy as np
from numpy import uint8

In [2]:
from tkinter import filedialog
from tkinter.filedialog import asksaveasfile
from tkinter import *
import tkinter.messagebox as msg
from PIL import *
import os
import tkinter as tk
from tkinter import ttk
from PIL import ImageTk,Image

In [3]:
def extractImgData(imgLoc):
    pixelMatrix = imageio.imread(imgLoc)
    return pixelMatrix

In [4]:
def grayscale(height, width, pixelMatrix):
    statusvar.set("Loading 0%")
    statusBar.update()
    for i in range(height):
        for j in range(width):
            avgCol = ((int)(pixelMatrix[i,j,0]) + (int)(pixelMatrix[i,j,1]) + (int)(pixelMatrix[i,j,2]))/3
            pixelMatrix[i,j,0] = pixelMatrix[i,j,1] = pixelMatrix[i,j,2] = uint8(avgCol)
        count = (int)(i*100/height)
        statusvar.set("Loading " + str(count) + "%")
        statusBar.update()
    statusvar.set("Ready Now")
    statusBar.update()
    time.sleep(3)
    statusvar.set("Status")

In [5]:
def vertical_flip(height, width, pixelMatrix):
    statusvar.set("Loading 0%")
    statusBar.update()
    i = 0
    while (i < height):
        j = 0
        while (j < width/2):
            redTemp = pixelMatrix[i,j,0]
            pixelMatrix[i,j,0] = pixelMatrix[i,width-1-j,0]
            pixelMatrix[i,width-1-j,0] = redTemp
            
            greenTemp = pixelMatrix[i,j,1]
            pixelMatrix[i,j,1] = pixelMatrix[i,width-1-j,1]
            pixelMatrix[i,width-1-j,1] = greenTemp
            
            blueTemp = pixelMatrix[i,j,2]
            pixelMatrix[i,j,2] = pixelMatrix[i,width-1-j,2]
            pixelMatrix[i,width-1-j,2] = blueTemp
            
            j += 1
        count = (int)(i*100/height)
        statusvar.set("Loading " + str(count) + "%")
        statusBar.update()
        i += 1

    statusvar.set("Ready Now")
    statusBar.update()
    time.sleep(3)
    statusvar.set("Status")

In [6]:
def horizontal_flip(height,width,pixelMatrix):
    statusvar.set("Loading 0%")
    statusBar.update()
    i = 0
    while (i < height/2):
        j = 0
        while (j < width):
            redTemp = pixelMatrix[i,j,0]
            pixelMatrix[i,j,0] = pixelMatrix[height-1-i,j,0]
            pixelMatrix[height-1-i,j,0] = redTemp
            
            greenTemp = pixelMatrix[i,j,1]
            pixelMatrix[i,j,1] = pixelMatrix[height-1-i,j,1]
            pixelMatrix[height-1-i,j,1] = greenTemp
            
            blueTemp = pixelMatrix[i,j,2]
            pixelMatrix[i,j,2] = pixelMatrix[height-1-i,j,2]
            pixelMatrix[height-1-i,j,2] = blueTemp
            
            j += 1
        count = (int)(i*100/height)
        statusvar.set("Loading " + str(count) + "%")
        statusBar.update()
        i += 1
    statusvar.set("Ready Now")
    statusBar.update()
    time.sleep(3)
    statusvar.set("Status")

In [7]:
def init_surr_cells(pixel):
    surr_cells = [[[0,0,0],[0,0,0],[0,0,0]],[[0,0,0],[pixel[0], pixel[1], pixel[2]],[0,0,0]],[[0,0,0],[0,0,0],[0,0,0]]]
    return surr_cells

In [8]:
def Sobel_Operator(surr_cells, color):
    GX = [[-1,0,1],[-2,0,2],[-1,0,1]]
    GY = [[-1,-2,-1],[0,0,0],[1,2,1]]
    val_gx = val_gy = 0
    if color == 'r':
        for i in range(3):
            for j in range(3):
                val_gx = val_gx + surr_cells[i][j][0]*GX[i][j]
                val_gy = val_gy + surr_cells[i][j][0]*GY[i][j]
    elif color == 'g':
        for i in range(3):
            for j in range(3):
                val_gx = val_gx + surr_cells[i][j][1]*GX[i][j]
                val_gy = val_gy + surr_cells[i][j][1]*GY[i][j]
    else:
        for i in range(3):
            for j in range(3):
                val_gx = val_gx + surr_cells[i][j][2]*GX[i][j]
                val_gy = val_gy + surr_cells[i][j][2]*GY[i][j]
    net_val = round(math.sqrt(val_gx**2 + val_gy**2))
    if net_val > 255:
        net_val = 255
    return net_val

In [9]:
def edgify(height,width,pixelMatrix):
    statusvar.set("Loading 0%")
    statusBar.update()
    temp = list()
    i = 0
    while i < height:
        j = 0
        while j < width:
            surr_cells = init_surr_cells(pixelMatrix[i,j])
            
            if i != height - 1:
                surr_cells[2][1][0] = pixelMatrix[i+1,j,0]
                surr_cells[2][1][1] = pixelMatrix[i+1,j,1]
                surr_cells[2][1][2] = pixelMatrix[i+1,j,2]
                if j != width - 1:
                    surr_cells[2][2][0] = pixelMatrix[i+1,j+1,0]
                    surr_cells[2][2][1] = pixelMatrix[i+1,j+1,1]
                    surr_cells[2][2][2] = pixelMatrix[i+1,j+1,2]
                if j != 0:
                    surr_cells[2][0][0] = pixelMatrix[i+1,j-1,0]
                    surr_cells[2][0][1] = pixelMatrix[i+1,j-1,1]
                    surr_cells[2][0][2] = pixelMatrix[i+1,j-1,2]
                 
            if i != 0:
                surr_cells[0][1][0] = pixelMatrix[i-1,j,0]
                surr_cells[0][1][1] = pixelMatrix[i-1,j,1]
                surr_cells[0][1][2] = pixelMatrix[i-1,j,2]
                if j != width - 1:
                    surr_cells[0][2][0] = pixelMatrix[i-1,j+1,0]
                    surr_cells[0][2][1] = pixelMatrix[i-1,j+1,1]
                    surr_cells[0][2][2] = pixelMatrix[i-1,j+1,2]
                if j != 0:
                    surr_cells[0][0][0] = pixelMatrix[i-1,j-1,0]
                    surr_cells[0][0][1] = pixelMatrix[i-1,j-1,1]
                    surr_cells[0][0][2] = pixelMatrix[i-1,j-1,2]
                 
            if j != width - 1:
                surr_cells[1][2][0] = pixelMatrix[i,j+1,0]
                surr_cells[1][2][1] = pixelMatrix[i,j+1,1]
                surr_cells[1][2][2] = pixelMatrix[i,j+1,2]
             
            if j != 0:
                surr_cells[1][0][0] = pixelMatrix[i,j-1,0]
                surr_cells[1][0][1] = pixelMatrix[i,j-1,1]
                surr_cells[1][0][2] = pixelMatrix[i,j-1,2]
            x = Sobel_Operator(surr_cells, 'r') 
            y = Sobel_Operator(surr_cells, 'g') 
            z = Sobel_Operator(surr_cells, 'b') 
            temp.append([x,y,z])
            j += 1
        count = (int)(i*100/height)
        statusvar.set("Loading " + str(count) + "%")
        statusBar.update()
        i += 1
    i = 0
    k = 0
    while i < height:
        j = 0
        while j < width:
            pixelMatrix[i,j,0] = (uint8)(temp[k][0])
            pixelMatrix[i,j,1] = (uint8)(temp[k][1])
            pixelMatrix[i,j,2] = (uint8)(temp[k][2])
            k += 1
            j += 1
        i += 1
    statusvar.set("Ready Now")
    statusBar.update()
    time.sleep(3)
    statusvar.set("Status")

In [10]:
def solarization(height, width, pixelMatrix, threshold):
    statusvar.set("Loading 0%")
    statusBar.update()
    i = 0
    while i < height:
        j = 0
        while j < width:
            if pixelMatrix[i,j,0] >= threshold:
                pixelMatrix[i,j,0] = 255 - pixelMatrix[i,j,0]
                
            if pixelMatrix[i,j,1] >= threshold:
                pixelMatrix[i,j,1] = 255 - pixelMatrix[i,j,1]
                
            if pixelMatrix[i,j,2] >= threshold:
                pixelMatrix[i,j,2] = 255 - pixelMatrix[i,j,2]
            j += 1
        count = (int)(i*100/height)
        statusvar.set("Loading " + str(count) + "%")
        statusBar.update()
        i += 1
    statusvar.set("Ready Now")
    statusBar.update()
    time.sleep(3)
    statusvar.set("Status")

In [11]:
def brightness(height, width, pixelMatrix,scalingFactor):
    statusvar.set("Loading 0%")
    statusBar.update()
    i=0
    while(i<height):
        j=0
        while(j<width):
            if(pixelMatrix[i,j,0]+scalingFactor>255):
                pixelMatrix[i,j,0]=255
            else:
                 pixelMatrix[i,j,0]=pixelMatrix[i,j,0]+scalingFactor

            if(pixelMatrix[i,j,1]+scalingFactor>255):
                pixelMatrix[i,j,1]=255
            else:
                pixelMatrix[i,j,1]=pixelMatrix[i,j,1]+scalingFactor

            if(pixelMatrix[i,j,2]+scalingFactor>255):
                pixelMatrix[i,j,2]=255
            else:
                pixelMatrix[i,j,2]=pixelMatrix[i,j,2]+scalingFactor
            j+=1
        count = (int)(i*100/height)
        statusvar.set("Loading " + str(count) + "%")
        statusBar.update()
        i+=1
    statusvar.set("Ready Now")
    statusBar.update()
    time.sleep(3)
    statusvar.set("Status")

In [12]:
def contrast(height,width, pixelMatrix, contrastFactor):
    statusvar.set("Loading 0%")
    statusBar.update()
    i=0
    F=(259*(255+contrastFactor))/(255*(259-contrastFactor))
    while(i<height):
        j=0
        while(j<width):
            if((F*(pixelMatrix[i,j,0]-128)) + 128 >255):
                pixelMatrix[i,j,0]=255
            elif((F*(pixelMatrix[i,j,0]-128)) + 128 <0):
                pixelMatrix[i,j,0]=0
            else:
                pixelMatrix[i,j,0]=(F*(pixelMatrix[i,j,0]-128)) + 128 
            if((F*(pixelMatrix[i,j,1]-128)) + 128 >255):
                pixelMatrix[i,j,1]=255
            elif((F*(pixelMatrix[i,j,1]-128)) + 128 <0):
                pixelMatrix[i,j,1]=0
            else:
                pixelMatrix[i,j,1]=(F*(pixelMatrix[i,j,1]-128)) + 128
            if((F*(pixelMatrix[i,j,2]-128)) + 128 > 255):
                pixelMatrix[i,j,2]=255
            elif((F*(pixelMatrix[i,j,2]-128)) + 128 <0):
                pixelMatrix[i,j,2]=0
            else:
                pixelMatrix[i,j,2]=(F*(pixelMatrix[i,j,2]-128)) + 128
            j+=1
        count = (int)(i*100/height)
        statusvar.set("Loading " + str(count) + "%")
        statusBar.update()
        i+=1
    statusvar.set("Ready Now")
    statusBar.update()
    time.sleep(3)
    statusvar.set("Status")

In [13]:
def blur(height, width, pixelMatrix, counter):
    statusvar.set("Loading 0%")
    statusBar.update()
    k = 0
    for x in range(counter):
        temp=list()
        i=0
        while(i<height):
            j=0
            while(j<width):
                avgR = (int)(pixelMatrix[i,j,0])
                avgG = (int)(pixelMatrix[i,j,1])
                avgB = (int)(pixelMatrix[i,j,2])
                totalNeighbors=1

                if(i!=height-1):
                    avgR += (int)(pixelMatrix[i+1,j,0])
                    avgG += (int)(pixelMatrix[i+1,j,1])
                    avgB += (int)(pixelMatrix[i+1,j,2])
                    totalNeighbors+=1

                    if(j!=width-1):
                        avgR += (int)(pixelMatrix[i+1,j+1,0])
                        avgG += (int)(pixelMatrix[i+1,j+1,1])
                        avgB += (int)(pixelMatrix[i+1,j+1,2])
                        totalNeighbors+=1

                    if(j!=0):
                        avgR += (int)(pixelMatrix[i+1,j-1,0])
                        avgG += (int)(pixelMatrix[i+1,j-1,1])
                        avgB += (int)(pixelMatrix[i+1,j-1,2])
                        totalNeighbors+=1

                if(i!=0):
                    avgR += (int)(pixelMatrix[i-1,j,0])
                    avgG += (int)(pixelMatrix[i-1,j,1])
                    avgB += (int)(pixelMatrix[i-1,j,2])
                    totalNeighbors+=1

                    if(j!=width-1):
                        avgR += (int)(pixelMatrix[i-1,j+1,0])
                        avgG += (int)(pixelMatrix[i-1,j+1,1])
                        avgB += (int)(pixelMatrix[i-1,j+1,2])
                        totalNeighbors+=1

                    if(j!=0):
                        avgR += (int)(pixelMatrix[i-1,j-1,0])
                        avgG += (int)(pixelMatrix[i-1,j-1,1])
                        avgB += (int)(pixelMatrix[i-1,j-1,2])
                        totalNeighbors+=1

                if(j!=width-1):
                    avgR += (int)(pixelMatrix[i,j+1,0])
                    avgG += (int)(pixelMatrix[i,j+1,1])
                    avgB += (int)(pixelMatrix[i,j+1,2])
                    totalNeighbors+=1

                if(j!=0):
                    avgR += (int)(pixelMatrix[i,j-1,0])
                    avgG += (int)(pixelMatrix[i,j-1,1])
                    avgB += (int)(pixelMatrix[i,j-1,2])
                    totalNeighbors+=1
                avgR = round(avgR/totalNeighbors)
                avgB = round(avgB/totalNeighbors)
                avgG = round(avgG/totalNeighbors)
                if (avgR > 255):
                    avgR = 255
                if (avgB > 255):
                    avgB = 255
                if (avgG > 255):
                    avgG = 255
                temp.append([avgR,avgG,avgB])
                j=j+1
            count = (int)(k*100/(counter*height))
            statusvar.set("Loading " + str(count) + "%")
            statusBar.update()
            i=i+1
            k=k+1

        f=0
        g=0
        l=0
        while(f<height):
            g=0
            while(g<width):
                pixelMatrix[f][g][0]=(uint8)(temp[l][0])
                pixelMatrix[f][g][1]=(uint8)(temp[l][1])
                pixelMatrix[f][g][2]=(uint8)(temp[l][2])
                l=l+1
                g=g+1
            f=f+1
    statusvar.set("Ready Now")
    statusBar.update()
    time.sleep(3)
    statusvar.set("Status")

In [14]:
def vignette(height,width,pixelMatrix):
    statusvar.set("Loading 0%")
    statusBar.update()
    kernelMatrix = cv2.getGaussianKernel(height,300)*cv2.getGaussianKernel(width,300).T
    normalizedKernelMatrix = kernelMatrix/np.linalg.norm(kernelMatrix)
    scale = 255
    maskMatrix = scale*normalizedKernelMatrix
    for x in range(3):
        pixelMatrix[:,:,x] = pixelMatrix[:,:,x]*maskMatrix
        count = (int)(x*100/3)
        statusvar.set("Loading " + str(count) + "%")
        statusBar.update()
    statusvar.set("Ready Now")
    statusBar.update()
    time.sleep(3)
    statusvar.set("Status")

In [15]:
root = tk.Tk()

root.title("Signals and Systems project [GROUP-18]")
root.geometry("1020x1080")
root.configure(background="#212529")

def imgPreview():
    global myimage
    global img1
    global pixelMatrix
    global height
    global width

    root.filename = filedialog.askopenfilename(initialdir="C:/",
                                               title="Open Image"
                                               , filetypes=(("jpg files", "*.jpg"), ("png files", "*.png") , ("all files", "*.*")))
    img1=Image.open(root.filename)
    img1.thumbnail((500,1080))
    myimage = ImageTk.PhotoImage(img1)
    canvas0.create_image(350, 300, anchor=CENTER, image=myimage)
    pixelMatrix = extractImgData(root.filename)
    height = len(pixelMatrix)
    width = len(pixelMatrix[0])

def ExportImgPreview():
    global myimage2
    global img2
    imageio.imsave("temp.png", pixelMatrix)
    root.filename = "temp.png"
    img2 = Image.open(root.filename)
    img2.thumbnail((500, 1080))
    myimage2 = ImageTk.PhotoImage(img2)
    canvas1.create_image(350, 300, anchor=CENTER, image=myimage2)

def saveimg():
    root.filename2 = filedialog.asksaveasfile(title="Save",
                                              filetypes=(('jpeg', '*.jpg'), ('png', '*.png'), ('all files', '*.*'))
                                              , defaultextension=(('jpeg', '*.jpg')))#, ('png', '*.png'), ('all files', '*.*')))
    imageio.imsave(root.filename2.name, pixelMatrix)
    
def help():    
    msg.showinfo("Help",'''FILTERS OF MINI-PHOTOSHOP:-
    
Brightness- Refers to overall lightness or darkness of the image.

Contrast- It shows the difference between dark and light in an image.

Solarization- The filter gives the tone reversal of the image which is often observed in photographic film in camera due to overexposure.

Blur- Smoothening of the image.

Grayscale- Grayscale is a range of shades of gray without apparent color.

Horizontal Flip- reverts the image along vertical axis

Vertical Flip- reverts the image along horizontal axis

Edge Detection- Make the edges visible

Vignette- Used for emphasizing central portion of the image and peripheries are made darker
''')

def info():
    msg.showinfo("Info","This Application was created under the ECE210 Signals and Systems project under the guidance of Professor Ashok Ranade. \n\nThis Application was developed by Shubham Jain, Gaurav Bajaj, Abrar Savan and Maanas Bhardwaj. \n\nThe main aim of the project is to create a mini-photoshop application that can be used to apply filters and various other features to an image. It helps users to modify an image as per the options available and later save the image in the device after execution.The main objectives of the project is to give users a wide range of filters and conversions in order to meet the needs of the user. The filters include grayscaling, brightness, blur, contrast, solarization,vignette, edge detection along with some features like horizontal and vertical reflection of image i.e. horizontal flip and vertical flip.")

mainmenu = Menu(root)
m1= Menu(mainmenu,tearoff=0)
m1.add_command(label="Open",command=lambda:imgPreview())
m1.add_command(label="Save",command=saveimg)

mainmenu.add_cascade(label="File", menu=m1)
root.config(menu= mainmenu)

m2= Menu(mainmenu,tearoff=0)
m2.add_command(label="Info",command=info)

mainmenu.add_cascade(label="Info", menu=m2)
root.config(menu= mainmenu)

m3= Menu(mainmenu,tearoff=0)
m3.add_command(label="Help",command=help)

mainmenu.add_cascade(label="Help", menu=m3)
root.config(menu= mainmenu)

#  ////    ROW 0
canvas0 = tk.Canvas(root, height=600, width=700, bg="#457b9d")
canvas0.grid(row=0,column=1, columnspan=2, rowspan=14, padx=0, pady=0, ipadx=0, ipady=43)

# button  openImage
openImageButton = tk.Button(root, text="OPEN IMAGE", padx=10, pady=0, fg="black", bg="#d3d3d3", command=lambda:imgPreview())
openImageButton.grid(row=17,column=1, rowspan=1, columnspan=1, sticky=NW, padx=35, pady=20, ipadx=0, ipady=5)
#print(height,width,pixelMatrix)

# button  VignetteButton
GradientButton = tk.Button(root, text="VIGNETTE", padx=10, pady=0, fg="white", bg="#FF8811",command=lambda:vignette(height,width,pixelMatrix))
GradientButton.grid(row=1, column=0, sticky=NW, padx=35, pady=0, ipadx=0, ipady=0)
# button  EdgeDetectionButton
EdgeDetectionButton = tk.Button(root, text="EDGE DETECTION", padx=10, pady=0, fg="white", bg="#FF8811", command=lambda:edgify(height,width,pixelMatrix))
#threading.Thread(target = lambda:edgify(height,width,pixelMatrix)).start()
EdgeDetectionButton.grid(row=2, column=0, sticky=NW, padx=35, pady=0, ipadx=0, ipady=0)
# button  GrayscaleButton
GrayscaleButton = tk.Button(root, text="GRAYSCALE", padx=10, pady=0, fg="white", bg="#FF8811",command=lambda:grayscale(height,width,pixelMatrix))
GrayscaleButton.grid(row=3, column=0, sticky=NW, padx=35, pady=0, ipadx=0, ipady=0)
# button  HorizontalFlipButton
HorizontalFlipButton = tk.Button(root, text="VERTICAL FLIP", padx=10, pady=0, fg="white", bg="#FF8811",command=lambda:horizontal_flip(height,width,pixelMatrix))
HorizontalFlipButton.grid(row=4, column=0, sticky=NW, padx=35, pady=0, ipadx=0, ipady=0)

#  ////  ROW 1

canvas1 =tk.Canvas(root,height=600, width=700, bg="#264653")
canvas1.grid(row=0,column=3, columnspan=2, rowspan=14, padx=0, pady=0, ipadx=0, ipady=43)
# button  VerticalFlipButton
VerticalFlipButton = tk.Button(root, text="HORIZONTAL FLIP", padx=10, pady=0, fg="white", bg="#FF8811",command=lambda:vertical_flip(height,width,pixelMatrix))
VerticalFlipButton.grid(row=5, column=0, sticky=NW, padx=35, pady=5, ipadx=0, ipady=0)

# slider  solarisation
slide_solarisation= Scale(root, from_=0, to=255, orient=HORIZONTAL)
slide_solarisation.grid(row=6, column=0, sticky=NW, padx=35, pady=20, ipadx=0, ipady=0)

# slider  brightness
slide_brightness= Scale(root, from_=1, to=100, orient=HORIZONTAL)
slide_brightness.grid(row=8, column=0, sticky=NW, padx=35, pady=20, ipadx=0, ipady=0)

# slider  contrast
slide_contrast= Scale(root, from_=1, to=100, orient=HORIZONTAL)
slide_contrast.grid(row=10, column=0, sticky=NW, padx=35, pady=20, ipadx=0, ipady=0)

# slider  blur
slide_blur= Scale(root, from_=1, to=4, orient=HORIZONTAL)
slide_blur.grid(row=12, column=0, sticky=NW, padx=35, pady=20, ipadx=0, ipady=0)

#Reset button
#ResetButton = tk.Button(root, text="RESET", padx=10, pady=0, fg="white", bg="#FF8811",command=lambda:reset(root.filename))
#ResetButton.grid(row=17, column=0, sticky=NW, padx=35, pady=5, ipadx=0, ipady=0)

label = Label(root, text= slide_solarisation.get())
slide_button = Button(root, text="SOLARIZATION",command=lambda:solarization(height,width,pixelMatrix,slide_solarisation.get()), padx=10, pady=5, fg="white", bg="#0096c7")
slide_button.grid(row=7, column=0, sticky="", padx=35, pady=0, ipadx=0, ipady=0)

label = Label(root, text= slide_brightness.get())
slide_button = Button(root, text="BRIGHTNESS",command=lambda:brightness(height,width,pixelMatrix,slide_brightness.get()), padx=10, pady=5, fg="white", bg="#0096c7")
slide_button.grid(row=9, column=0, sticky="", padx=0, pady=0, ipadx=0, ipady=0)

label = Label(root, text= slide_contrast.get())
slide_button = Button(root, text="CONTRAST",command=lambda:contrast(height,width,pixelMatrix,slide_contrast.get()), padx=10, pady=5, fg="white", bg="#0096c7")
slide_button.grid(row=11, column=0, sticky="", padx=0, pady=0, ipadx=0, ipady=0)

label = Label(root, text= slide_blur.get())
slide_button = Button(root, text="BLUR",command=lambda:blur(height,width,pixelMatrix,slide_blur.get()), padx=10, pady=5, fg="white", bg="#0096c7")
slide_button.grid(row=13, column=0, sticky="", padx=0, pady=0, ipadx=0, ipady=0)

# button 8 Export
Export = tk.Button(root, text="EXPORT", padx=0, pady=0, fg="black", bg="#d3d3d3", command=saveimg)
Export.grid(row=17, column=2, sticky="", padx=14, pady=20, ipadx=5, ipady=5)

# //// ROW 7
# button 9 Exit
Exit = tk.Button(root, text="EXIT",padx=0, pady=0, fg="black", bg="#d3d3d3", command=root.destroy)
Exit.grid(row=17, column=4, sticky="", padx=0, pady=20, ipadx=0, ipady=5)

#  button 10 Preview
preview = Button(root, text="PREVIEW", command=ExportImgPreview,padx=0, pady=0, fg="black", bg="#d3d3d3")
preview.grid(row=17, column=3, sticky="", padx=0, pady=20, ipadx=0, ipady=5)

#Status Bar
global statusvar
global statusBar
statusvar= StringVar()
statusvar.set("Status")
statusBar=Label(root,textvariable= statusvar, relief=SUNKEN,anchor="w")
statusBar.grid(row=18, columnspan=5, sticky="ew", padx=0, pady=0, ipadx=0, ipady=0)

root.mainloop()
img1.close()
img2.close()
os.remove("temp.png")